In [1]:
import os

In [2]:
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import io


In [5]:

app = Flask(__name__)
model = load_model('eye_disease_model.h5')


# Define the directory where uploaded images will be stored
UPLOAD_FOLDER = 'uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Function to preprocess the uploaded image
def preprocess_image(image):
    # Resize the image to match the input size expected by your model
    resized_image = image.resize((222, 222))
    # Convert image to numpy array
    img_array = np.array(resized_image)
    # Normalize pixel values
    img_array = img_array / 255.0
    # Expand dimensions to create a batch for the model
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to convert model predictions to class labels
def get_class_label(prediction):
    classes = ['Cataract', 'Diabetic Retinopathy', 'Glaucoma', 'Normal']
    predicted_class = np.argmax(prediction)
    return classes[predicted_class]

@app.route('/')
def index():
    return render_template('index2.html')

@app.route('/classify', methods=['POST'])
def classify():
    # Check if a file was uploaded
    if 'image' not in request.files:
        return render_template('index2.html', error='No file uploaded')

    # Get the uploaded image file
    uploaded_file = request.files.get('image')

    # If the user does not select a file, browser also submit an empty part without filename
    if uploaded_file.filename == '':
        return render_template('index2.html', error='No selected file')

    # Read the image file
    img = Image.open(uploaded_file.stream)

    # Preprocess the image
    processed_img = preprocess_image(img)
# Save the uploaded file
    filename = os.path.join(app.config['UPLOAD_FOLDER'], uploaded_file.filename)
    uploaded_file.save(filename)

    # Make prediction using the model
    prediction = model.predict(processed_img)

    # Get class label
    result = get_class_label(prediction[0])

    return render_template('result.html', result=result)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1